In [ ]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LeakyReLU
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
sol = pd.read_csv('solutions_week2.csv', sep=';').iloc[:,[0,-1]]
solutions = [json.loads(i) for i in sol['solution']]
sol['solution'] = solutions

sol5 = pd.read_csv('week5_small_sol.csv', sep=';').iloc[:,[0,-1]]
solutions = [json.loads(i) for i in sol5['solution']]
sol5['solution'] = solutions

sol = sol.append(sol5)
sol

In [ ]:
vert = pd.read_csv('nodes_v5.csv', index_col=None)

deleted = []
c = []
for r in tqdm(vert.iterrows(), total=len(vert)):
    data = r[1]
    s = sol[sol['instance'] == data['instance']]['solution'].to_numpy()[0]
    if data['nodeId'] in s:
        deleted.append(1)
    else:
        deleted.append(0)
        
vert['y'] = deleted


vert.to_csv('vertices_week2_v6_with_deleted.csv', index=False)
vert

In [ ]:
sum(vert['y'])

In [ ]:
X = vert.iloc[:,3:-1].to_numpy()
y = vert['y'].to_numpy(dtype=np.float64)

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.1)

In [ ]:
model = Sequential()
model.add(Dense(12, input_dim=12))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.1))
model.add(Dense(12, input_dim=12))
model.add(Dropout(0.05))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(6, input_dim=6))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, y_train,validation_data = (X_test,y_test), epochs=175, batch_size=64);

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# deleted = 1, not-deleted = 0

x = X_test
Y = y_test

pred = [int(i[0] > 0.7) for i in model.predict(x, verbose=False)]
delC = 0;
delF = 0;
NdelC = 0;
NdelF = 0;

for i in range(len(Y)):
    if Y[i] == 1 and pred[i] == 1:
        delC += 1
    elif Y[i] == 1 and pred[i] == 0:
        delF += 1
    elif Y[i] == 0 and pred[i] == 0:
        NdelC += 1
    elif Y[i] == 0 and pred[i] == 1:
        NdelF += 1
        
print("Act\Pred| Keep\t| Del\nKeep \t|",NdelC,"\t|", NdelF, "\nDel\t|", delF,"\t|", delC)

In [ ]:
# Writes the model to a file which can be interpreted by the Java solver
w = model.get_weights()
with open('mat_v6.txt', 'w') as f:
    for layer in w:
        if layer.ndim == 2:    # Weight layer
            lay = layer.T
            f.write(';\n')
            for row in lay:
                f.write(str(list(row)).replace('[', '').replace(']', '') + '\n')
            f.write(';\n')
        if layer.ndim == 1:    # Bias layer
            f.write(str(list(layer)).replace('[', '').replace(']', '') + '\n')